In [1]:
import aiida

# must execute it in the first place
aiida.load_profile()

In [2]:
from aiida.manage.configuration import get_profile

from aiida.common.datastructures import CalcInfo, CodeInfo
from aiida.common.folders import Folder

from aiida.engine import CalcJob, WorkChain
from aiida.engine import calcfunction, workfunction, submit, run

from aiida.plugins import DataFactory, WorkflowFactory

from aiida.parsers.parser import Parser
from aiida.orm import Code
from aiida.orm import load_code, load_node
from aiida.orm import Str, Dict, Float, Int, Bool
from aiida.orm import QueryBuilder

import aiida

from alamode_aiida.data_loader import load_anphon_kl, load_anphon_kl_spec
from ase.io.espresso import write_espresso_in
from ase.io.lammpsdata import read_lammps_data
from ase.build import make_supercell
from ase import Atom, Atoms
import ase

from itertools import combinations_with_replacement
import numpy as np
import os
import subprocess
import shutil
import re
from time import sleep
import spglib
import json

from os.path import expanduser

from tools import NodeBank
from tools import wait_for_node_finished
from alamode_aiida.ase_support import load_atoms
from tools.putstructure import PutStructure


In [3]:
# load types
StructureData = DataFactory('structure')
FolderData = DataFactory('folder')
SinglefileData = DataFactory('singlefile')
ArrayData = DataFactory('array')
List = DataFactory('list')


In [4]:
g_graph = False

with open("setting.json") as f:
    _dic = json.load(f)
print(_dic)
CWD = _dic["CWD"]
_path = os.path.abspath(CWD)
os.makedirs(_path, exist_ok=True)
_path

{'CWD': 'run29'}


'/home/max/Documents/aiida-test/alamode-aiida/example/run29'

In [5]:
# 再実行時に作成したノードを用いるためにnodebankを使う。
g_force = False
nodebank = NodeBank(_path, force=g_force)

In [6]:
# 計算するroot directoryの定義
key = "root_wd"
g_root_wd = nodebank.load_or_dump(key, Str(os.path.join(os.getcwd(), CWD)))
os.makedirs(g_root_wd.value, exist_ok=True)
g_root_wd


debug, node uuid: ce1542b4-8c75-4a89-8ecc-eef6dac75026 (unstored) value: /home/max/Documents/aiida-test/alamode-aiida/example/run29
debug, load None
debug, force dump


<Str: uuid: ce1542b4-8c75-4a89-8ecc-eef6dac75026 (pk: 59093) value: /home/max/Documents/aiida-test/alamode-aiida/example/run29>

In [7]:
# 構造ファイルの取得
# lammpsから取得する場合の例
# lammpsで最適化しているはずなのでsimulatorもLAMMPSを用いる。

crystal = "GaN222"
nodebank.load_or_dump("cyrstal", Str(crystal)) 

if crystal == "GaN222":
    _path = os.path.abspath("lammps_input/GaN/GaN222.lammps")
    g_diag = ArrayData()
    # supercellをファイルの構造の何倍にするかの定義をする   
    g_diag.set_array('diag', np.array([1, 1, 1]))

elif crystal == "Si222":
    _path = os.path.abspath("lammps_input/Si222/Si222.lammps")
    g_diag = ArrayData()
    # supercellをファイルの構造の何倍にするかの定義をする   
    g_diag.set_array('diag', np.array([1, 1, 1]))
else:
    raise ValueError('unknow crystal.')
        
g_diag = nodebank.load_or_dump("diag", g_diag)    
g_structure_org_filename = Str(_path)
g_file_format = Str("lammps-data")
nodebank.dump("structure_org_format", g_file_format)
g_file_style = Str("atomic")
nodebank.dump("structure_org_style", g_file_style)
g_supply_Z = Bool(True)
nodebank.dump("supply_Z", g_supply_Z)
g_raw_structure = load_atoms(
    g_structure_org_filename, g_file_format, g_file_style, g_supply_Z)
g_raw_structure.get_ase()

debug, node uuid: 79bc5764-2468-42eb-a1d6-6dd409362fed (unstored) value: GaN222
debug, load None
debug, force dump
debug, node uuid: f6cae774-40b7-47fe-b1d0-193a9567d5ce (unstored)
debug, load None
debug, force dump


Atoms(symbols='Ga2N2Ga2N2Ga2N2Ga2N2Ga2N2Ga2N2Ga2N2Ga2N2', pbc=True, cell=[[6.38, 0.0, 0.0], [-3.19, 5.5252420761447, 0.0], [0.0, 0.0, 10.378]], masses=...)

In [8]:
g_file_format = Str("lammps-data")
g_primcell_filename = Str(f"primcell.lammps")
g_supercell_filename = Str(f"supercell.lammps")
inputs = {"cwd": g_root_wd, "diag": g_diag, "output_format": g_file_format,
          "primcell_filename": g_primcell_filename,
          "supercell_filename": g_supercell_filename,
          "structure": g_raw_structure}
g_primsuper_structures = run(PutStructure, **inputs)


prim Atoms(symbols='Ga2N2', pbc=False, cell=[[3.189999999999996, 0.0, 0.0], [-1.594999999999998, 2.7626210380723557, 0.0], [0.0, 0.0, 5.189]], masses=...)
atoms Atoms(symbols='Ga2N2', pbc=False, cell=[[3.19, 0.0, 0.0], [-1.595, 2.7626210380724, 0.0], [0.0, 0.0, 5.189]], masses=...)
path uuid: 3023dc70-1157-4610-8854-54280d391027 (pk: 59109) value: /home/max/Documents/aiida-test/alamode-aiida/example/run29/primcell.lammps
format uuid: 02b83567-bbd3-4e5b-8e83-2a1cc3569fc6 (pk: 59102) value: lammps-data
atoms Atoms(symbols='Ga2N2Ga2N2Ga2N2Ga2N2Ga2N2Ga2N2Ga2N2Ga2N2', pbc=True, cell=[[6.38, 0.0, 0.0], [-3.19, 5.5252420761447, 0.0], [0.0, 0.0, 10.378]], masses=...)
P [[1 0 0]
 [0 1 0]
 [0 0 1]]
super Atoms(symbols='Ga2N2Ga2N2Ga2N2Ga2N2Ga2N2Ga2N2Ga2N2Ga2N2', pbc=True, cell=[[6.38, 0.0, 0.0], [-3.19, 5.5252420761447, 0.0], [0.0, 0.0, 10.378]], masses=...)
atoms Atoms(symbols='Ga2N2Ga2N2Ga2N2Ga2N2Ga2N2Ga2N2Ga2N2Ga2N2', pbc=True, cell=[[6.38, 0.0, 0.0], [-3.19, 5.5252420761447, 0.0], [0.0, 0.0, 

In [9]:
g_primsuper_structures  # outout

{'primstructure_filepath': <Str: uuid: 3023dc70-1157-4610-8854-54280d391027 (pk: 59109) value: /home/max/Documents/aiida-test/alamode-aiida/example/run29/primcell.lammps>,
 'superstructure_filepath': <Str: uuid: 7afe9cad-2048-4d4a-90a2-f729ea7fa388 (pk: 59107) value: /home/max/Documents/aiida-test/alamode-aiida/example/run29/supercell.lammps>,
 'primstructure': <StructureData: uuid: f222a7ce-53e6-4d9d-b868-6b57cd8a2c92 (pk: 59111)>,
 'primstructure_file': <SinglefileData: uuid: 44c76374-8e88-47fe-90d1-f49f1d2d43a2 (pk: 59113)>,
 'superstructure': <StructureData: uuid: 133fed56-6fda-4abf-b706-e09bb5ffbde2 (pk: 59115)>,
 'superstructure_file': <SinglefileData: uuid: 28128ea5-41f8-4dd5-a101-fb575732c817 (pk: 59117)>}

In [10]:
# filepath
g_primstructure_filepath = g_primsuper_structures["primstructure_filepath"]
# filepath
g_superstructure_filepath = g_primsuper_structures["superstructure_filepath"]

In [11]:
g_superstructure = g_primsuper_structures["superstructure"]  # StructureData
g_primstructure = g_primsuper_structures["primstructure"]  # StructureData

In [12]:
# StructureData
g_superstructure_file = g_primsuper_structures["superstructure_file"]

In [13]:
if False:
    pk = g_ciffilename.pk
    print(pk)
    !verdi node graph generate $pk

In [14]:
nodebank.dump("primstructure_filepath", g_primstructure_filepath)
nodebank.dump("superstructure_filepath", g_superstructure_filepath)
nodebank.dump("superstructure_file", g_superstructure_file)
nodebank.dump("primstructure", g_primstructure)
nodebank.dump("superstructure", g_superstructure)

<StructureData: uuid: 133fed56-6fda-4abf-b706-e09bb5ffbde2 (pk: 59115)>

In [15]:
_, g_primstructure_filename = os.path.split(g_primstructure_filepath.value)
nodebank.dump("superstructure_filename", Str(g_primstructure_filename))

_, g_superstructure_filename = os.path.split(g_superstructure_filepath.value)
nodebank.dump("superstructure_filename", Str(g_superstructure_filename))

<Str: uuid: c60d5c7e-a3c8-4b16-8707-f5fc9ed8a75b (pk: 59119) value: supercell.lammps>

In [16]:
g_prefix = Str('harmonic')
nodebank.dump("prefix", g_prefix)
g_norder = Int(1)
nodebank.dump(f"{g_prefix.value}_norder", g_norder)

<Int: uuid: ea668f08-633f-4858-9bb2-7bd1fbfe8d0a (pk: 59121) value: 1>

In [17]:
# ファイルを置くsubdirectory名の作成
# ディレクトリ名 cwd=formula_supercell_harmonic
# ディレクトリがなければ作成する。
g_almprefix = "_".join([g_primstructure.get_formula(), "x".join(
    map(str, g_diag.get_array("diag"))), g_prefix.value])
g_almprefix = Str(g_almprefix)
g_cwd = Str(os.path.join(g_root_wd.value, g_almprefix.value))
g_cwd = nodebank.load_or_dump("cwd", g_cwd)
os.makedirs(g_cwd.value, exist_ok=True)
g_cwd

debug, node uuid: 7697e409-d2f7-4115-ab2e-63bbadf87ce6 (unstored) value: /home/max/Documents/aiida-test/alamode-aiida/example/run29/Ga2N2_1x1x1_harmonic
debug, load None
debug, force dump


<Str: uuid: 7697e409-d2f7-4115-ab2e-63bbadf87ce6 (pk: 59122) value: /home/max/Documents/aiida-test/alamode-aiida/example/run29/Ga2N2_1x1x1_harmonic>

In [18]:
nodebank.dump(f"{g_prefix.value}_almprefix", g_almprefix)

<Str: uuid: 515c8f86-755c-4a1a-a70f-4c994ab22bbd (pk: 59123) value: Ga2N2_1x1x1_harmonic>

In [19]:
g_superstructure.get_ase().get_scaled_positions().shape

(32, 3)